In [ ]:
import pandas as pd
import numpy as np
import re
import lxml

from bs4 import BeautifulSoup
from requests import get

%matplotlib inline

In [ ]:
url = 'https://www.imdb.com/title/tt0111161/'
response = get(url)
#print(response.text[:500])

In [ ]:
soup = BeautifulSoup(response.content, 'lxml') 

In [ ]:
print(type(soup.find(class_="subtext").contents[3].text))
print(type(soup.find(class_="subtext").contents[3].string))

In [ ]:
print(type(soup.find(class_="subtext").stripped_strings))
print(type(soup.find(class_="subtext").descendants))
print(type(soup.find(class_="subtext").children))
print(type(soup.find(class_="subtext").contents))

In [ ]:
tag = soup.find(id='titleDetails')
for child in tag.stripped_strings:
    chil = child
    print('Child:' ,child ,type(child))


In [ ]:
save = 0
val = ''
budget = ''
opening = ''
gross_usa = ''
gross_world = ''
release_date = ''

tag = soup.find(id='titleDetails')
for child in tag.stripped_strings:
    if val == 'Budget:':
        budget = child
        val = ''
    elif val == 'Release Date:':
        release_date = child
        val = ''
    elif val == 'Gross USA:':
        gross_usa = child
        val = ''
    elif val == 'Opening Weekend USA:':
        opening = child
        val = ''
    elif val == 'Cumulative Worldwide Gross:':
        gross_world = child
        val = ''
    print('Child:' ,child ,type(child) ,val ,'\n' ,' : ' ,save ,' : ' ,budget 
          ,' : ' ,opening ,' : ' ,gross_usa ,' : ' ,gross_world ,' : \n')
    if pd.Series(child).isin(['Release Date:','Budget:','Opening Weekend USA:','Gross USA:'
                   ,'Cumulative Worldwide Gross:']).bool():
        val = child
    else:
        val = ''
    print('Child:' ,child ,type(child) ,val ,'\n\n')

print(' : ' ,release_date ,' : ' ,budget ,' : ' ,opening ,' : ' ,gross_usa ,' : ' ,gross_world ,' : ')

In [ ]:
title_wrapper = soup.find('div',class_="title_wrapper")
imdbRating = soup.find(class_="imdbRating")

In [ ]:
imdbRating_stripped = imdbRating.stripped_strings
review_count = list(imdbRating_stripped)[3]
review_count

In [ ]:
imdbRating_stripped = imdbRating.stripped_strings
user_review_count = list(imdbRating_stripped)[4]
user_review_count

In [ ]:
imdbRating_stripped = imdbRating.stripped_strings
critic_review_count = list(imdbRating_stripped)[5]
critic_review_count

In [ ]:
#to return error
forceError['doesnt_exist']

In [36]:
import pandas as pd
import numpy as np
import re
import lxml

from bs4 import BeautifulSoup
from requests import get

%matplotlib inline

In [37]:
import os
path=os.getcwd()
datapath="datasets"

In [38]:
def Timenow():
    from datetime import datetime
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)

In [39]:
#dtyped basics dataframe
def Basics_dtype():
    imdb_basics_raw=pd.read_csv(os.path.join(path,datapath,"imdb/title.basics.tsv/title.basics.tsv"), sep='\t', header=0)                                   
    #imdb_basics_copy=imdb_basics_raw.copy()
    col_basics = imdb_basics_raw.dtypes.to_dict()
    col_basics["tconst"]='object'
    col_basics['titleType']='category'
    col_basics['primaryTitle']='object'
    col_basics['originalTitle']='object'
    col_basics['isAdult']='bool'
    col_basics['startYear']='int64'
    col_basics['endYear']='int64'
    col_basics['runtimeMinutes']='int64'
    col_basics['genres']='object'
    #col_basics
    
    #imdb_basics_dtyped = imdb_basics.copy()
    #imdb_basics_dtyped['isAdult'] = imdb_basics_dtyped['isAdult'].astype('bool')
    imdb_basics_raw[['runtimeMinutes','startYear','endYear']] \
        = imdb_basics_raw[['runtimeMinutes','startYear','endYear']].apply(lambda x: \
            np.where((x=='\\N') | ~x.astype('str').str.isalnum() ,'0' ,x) \
            ,axis=0)
    imdb_basics_dtyped = imdb_basics_raw.astype(col_basics)
    imdb_basics_cols = ['tconst','titleType','primaryTitle','isAdult' \
                        ,'startYear','endYear','runtimeMinutes','genres']
    imdb_basics_dtyped = imdb_basics_dtyped[imdb_basics_cols].set_index(keys='tconst')
    imdb_basics_dtyped = imdb_basics_dtyped.loc[imdb_basics_dtyped['titleType']!='tvEpisode']
#feature expansion 
    imdb_basics_dtyped ['runtime_cat'] = pd.cut(imdb_basics_dtyped.runtimeMinutes
       ,bins=[-1 ,30 ,60 ,90 ,120 ,180 ,np.inf]
       ,labels=['0-30' ,'30-60' ,'60-90' ,'90-120' ,'120-180', 'Long'])
    
    return imdb_basics_dtyped

In [40]:
#feature expansion
def Basics_genre_expand(dfX):
    df07 = pd.DataFrame(dfX.genres.str.split(',',expand=True)) \
        .rename(columns=lambda x: x+1).add_prefix('genre')
    
    df08 = df07.stack().reset_index()[0]
    df0freq = df08.value_counts()
    cutoff = sum(df0freq)*0.01
    df0infreq = df0freq[df0freq<cutoff].index
    df010 = df08.replace(df0infreq ,'Other')
    df011 = list(df010.unique())
    df0col = pd.DataFrame(columns=df011).astype('bool')
    
    df020 = dfX['genres'].replace(df0infreq ,'Other')
    df01concat = [df020 ,df0col]
    df01uniq = pd.concat(df01concat ,axis=1)
    
    return df01uniq ,df011

In [41]:
#exclusively called by Genre_1hot
def Match_genre(dfX ,x ,Xcol ,col): #axis=index
    colname = x.name
    i = list(dfX.index)
    colval = dfX[col].loc[i]
    #colval = df.region.loc[i]
    return colval.apply(lambda f: colname in f)

In [43]:
#final genre pipeline
def Genre(dfX):
    try:
        dfY = basics[ratings.index.notnull()]
        basics ,basicscols = Basics_genre_expand(dfY)
        dfY = Genre_1hot(basics ,basicscols ,'genres')
    except:
        print('WENT TO EXCEPT 1')
        basics ,basicscols = Basics_genre_expand(dfX)
        dfY = Genre_1hot(basics ,basicscols ,'genres')
    
    return dfY

In [44]:
Timenow()

Current Time = 01:06:49


/Volumes/lin/MachineLearning/env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,titleType,primaryTitle,isAdult,startYear,endYear,runtimeMinutes,genres,runtime_cat
tconst,,,,,,,,
tt0000001,short,Carmencita,False,1894,0,1,"Documentary,Short",0-30
tt0000002,short,Le clown et ses chiens,False,1892,0,5,"Animation,Short",0-30
tt0000003,short,Pauvre Pierrot,False,1892,0,4,"Animation,Comedy,Romance",0-30
tt0000004,short,Un bon bock,False,1892,0,0,"Animation,Short",0-30
tt0000005,short,Blacksmith Scene,False,1893,0,1,"Comedy,Short",0-30
tt0000006,short,Chinese Opium Den,False,1894,0,1,Short,0-30
tt0000007,short,Corbett and Courtney Before the Kinetograph,False,1894,0,1,"Short,Sport",0-30
tt0000008,short,Edison Kinetoscopic Record of a Sneeze,False,1894,0,1,"Documentary,Short",0-30
tt0000009,movie,Miss Jerry,False,1894,0,45,Romance,30-60


In [11]:
Timenow()

Current Time = 20:19:22


/Volumes/lin/MachineLearning/env/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,genres,Documentary,Short,Animation,Comedy,Romance,Other,Action,Drama,Fantasy,...,Biography,Music,Crime,Family,Adventure,Mystery,\N,Sci-Fi,Thriller,Adult
tt0000001,"Documentary,Short",True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
tt0000002,"Animation,Short",False,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
tt0000003,"Animation,Comedy,Romance",False,False,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
tt0000004,"Animation,Short",False,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
tt0000005,"Comedy,Short",False,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
tt0000006,Short,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
tt0000007,"Short,Sport",False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
tt0000008,"Documentary,Short",True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
tt0000009,Romance,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
tt0000010,"Documentary,Short",True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [13]:
Timenow()

Current Time = 20:21:19


In [45]:
#exclusively called by Netflix_uniq0() ,Netflix_uniq1()
def Netflix_titles():
    netflix = pd.read_csv(os.path.join(path,datapath,"netflix_titles.csv"))
    
    netflix_uniqueTitles = pd.DataFrame(netflix.title.unique() ,columns=['title'])
    netflix_uniqueTitles.set_index(keys=['title'] ,inplace=True)
    
    return netflix_uniqueTitles

In [91]:
basics

""
tconst
tt0000001
tt0000002
tt0000003
tt0000004
tt0000005
tt0000006
tt0000007
tt0000008
tt0000009


In [46]:
#returns only index column
def Netflix_uniq0(dfX ,colX):
    netflix_uniqueTitles = Netflix_titles()
    
    indX = dfX.index.name
    dfX.reset_index(inplace=True)
    dfX.set_index(keys=colX ,inplace=True)
    dfX_titles = dfX.join(netflix_uniqueTitles ,how='inner')
    dfX_titles.reset_index(inplace=True)
    dfX_uniq = dfX_titles[indX].unique()
    dfX_uniq2 = pd.DataFrame(dfX_uniq ,columns=[indX]) 
    dfX_uniq2.set_index(keys=indX ,inplace=True ,drop=True)
    
    return dfX_uniq2

In [12]:
Timenow()

Current Time = 19:16:13


In [47]:
#exclusively called by NetflixStats()
#returns dfX but filtered by netflix
def Netflix_uniq1(dfX):
    indX = dfX.index.name
    
    try:
        imdb_basics = basics.reindex(ratings.index)
    except:
        print('WENT TO EXCEPT 2')
        imdb_basics = Basics_dtype()
        
    #indY = imdb_basics.index.name
    basics_netflix_uniq = Netflix_uniq0(imdb_basics ,'primaryTitle') 
    
    #basics_netflix_uniq.set_index(keys='tconst' ,inplace=True)
    #dfX_titles2 = dfX.join(basics_netflix_uniq ,how='inner')
    #dfX_titles2.drop(dfX_titles2.columns ,axis='columns' ,inplace=True)
    
    dfX_titles2 = dfX.reindex(basics_netflix_uniq.index)
    
    return dfX_titles2

#used to filter dataset based on netflix statistics in order to reduce imdb dataset to only number of votes
#where netflix data is applicable. goal of higher correlation of targeted imdb dataset.

In [48]:
#exclusively called by Ratings_netflix_filter
def NetflixStats(dfX):
    try:
        dfX_titles2 = dfX.reindex(ratings.index)
    except:
        print('WENT TO EXCEPT 3')
        ratings_netflix_uniq = Netflix_uniq1(dfX)
        
        #ratings_netflix = dfX.join(ratings_netflix_uniq ,how='inner' ,rsuffix="_ratings")
        #ratings_netflix_uniq.join(basics_netflix_uniq ,how='inner')
        #results in only 16436 rows × 0 columns

        dfX_titles2 = dfX.reindex(ratings_netflix_uniq.index)

    return pd.concat([dfX_titles2.mean()] + [dfX_titles2.std()])

In [45]:
basics

,tconst,titleType,isAdult,startYear,endYear,runtimeMinutes,genres,runtime_cat
primaryTitle,,,,,,,,
A Trip to the Moon,tt0000417,short,False,1902.0,0.0,13.0,"Action,Adventure,Comedy",0-30
The Great Train Robbery,tt0000439,short,False,1903.0,0.0,11.0,"Action,Crime,Short",0-30
The Birth of a Nation,tt0004972,movie,False,1915.0,0.0,195.0,"Drama,History,War",Long
Intolerance: Love's Struggle Throughout the Ages,tt0006864,movie,False,1916.0,0.0,163.0,"Drama,History",120-180
The Cabinet of Dr. Caligari,tt0010323,movie,False,1920.0,0.0,76.0,"Fantasy,Horror,Mystery",60-90
The Kid,tt0012349,movie,False,1921.0,0.0,68.0,"Comedy,Drama,Family",60-90
Nosferatu,tt0013442,movie,False,1922.0,0.0,94.0,"Fantasy,Horror",90-120
Safety Last!,tt0014429,movie,False,1923.0,0.0,74.0,"Action,Comedy,Thriller",60-90
Sherlock Jr.,tt0015324,movie,False,1924.0,0.0,45.0,"Action,Comedy,Romance",30-60


In [46]:
basics.join(Netflix_titles() ,how='inner')

,tconst,titleType,isAdult,startYear,endYear,runtimeMinutes,genres,runtime_cat
13 Reasons Why,tt1837492,tvSeries,False,2017.0,0.0,60.0,"Drama,Mystery",30-60
13 Sins,tt2059171,movie,False,2014.0,0.0,93.0,"Horror,Thriller",90-120
16 Blocks,tt0450232,movie,False,2006.0,0.0,102.0,"Action,Crime,Drama",90-120
1922,tt6214928,movie,False,2017.0,0.0,102.0,"Crime,Drama,Horror",90-120
2 States,tt2372678,movie,False,2014.0,0.0,149.0,"Comedy,Drama,Romance",120-180
20th Century Women,tt4385888,movie,False,2016.0,0.0,119.0,"Comedy,Drama",90-120
21,tt0478087,movie,False,2008.0,0.0,123.0,"Crime,Drama,History",120-180
21 & Over,tt1711425,movie,False,2013.0,0.0,93.0,Comedy,90-120
3 Idiots,tt1187043,movie,False,2009.0,0.0,170.0,"Comedy,Drama",120-180
3%,tt4922804,tvSeries,False,2016.0,0.0,49.0,"Action,Drama,Sci-Fi",30-60


In [49]:
#dtyped ratings dataframe
def Ratings_dtype():
    imdb_ratings_raw=pd.read_csv(os.path.join(path,datapath,"imdb/title.ratings.tsv/title.ratings.tsv"), sep='\t', header=0)           

    col_ratings = imdb_ratings_raw.dtypes.to_dict()
    col_ratings["tconst"]='object'
    col_ratings['averageRating']='float64'
    col_ratings['numVotes']='float64'
    
    imdb_ratings_dtyped = imdb_ratings_raw.astype(col_ratings)
    imdb_ratings_dtyped = imdb_ratings_dtyped[col_ratings].set_index(keys='tconst')  
    imdb_ratings_dtyped.dropna(inplace=True)
    
    return imdb_ratings_dtyped

In [50]:
#filters out by number of votes
def Ratings_netflix_filter(dfX): #Ratings_dtype()
    dfX_netflix_uniq = Netflix_uniq1(dfX)
    netflix_stats = NetflixStats(dfX_netflix_uniq)
    
    imdb_ratings_filtered = dfX.loc[dfX.numVotes > netflix_stats[1] - 0.0*netflix_stats[3]]
    
    return imdb_ratings_filtered

In [51]:
#filters out by number of votes
def Ratings_netflix_filter(dfX ,ctrl): #Ratings_dtype()
    if ctrl == 0:
        '''dfY = Ratings_dtype()
        ratings_netflix_uniq = Netflix_uniq1(dfY)
        netflix_stats = NetflixStats(ratings_netflix_uniq)
        imdb_ratings_filtered = dfY.loc[dfY.numVotes > netflix_stats[1] - 0.0*netflix_stats[3]]
        imdb_ratings_filtered(drop=imdb_ratings_filtered.columns ,axis='columns',inplace=True)   
        dfX_filtered = dfX.join(imdb_ratings_filtered ,how='inner')'''
        
        dfX_filtered = dfX.loc[ratings.index]
        
        return dfX_filtered
    else:
        ratings_netflix_uniq = Netflix_uniq1(dfX)
        netflix_stats = NetflixStats(ratings_netflix_uniq)
        
        imdb_ratings_filtered = dfX.loc[dfX.numVotes > netflix_stats[1] - 0.0*netflix_stats[3]]
    
        return imdb_ratings_filtered

In [52]:
#feature expansion
def Ratings_expand(dfX):
    imdb_ratings_filtered = dfX    #imdb_ratings_filtered = ratings_filtered(ratings_dtyped())
    
    rate_stats = pd.concat([dfX.mean()] + [dfX.std()])   #rate_stats
    
    imdb_ratings_filtered.loc[:,'ratecat'] \
        = imdb_ratings_filtered[['averageRating']].apply(lambda x: \
            sum([(x>rate_stats[0] - 2*rate_stats[2]) \
                ,(x>rate_stats[0] - rate_stats[2]) \
                ,(x>rate_stats[0]) \
                ,(x>rate_stats[0] + rate_stats[2]) \
                ,(x>rate_stats[0] + 2*rate_stats[2])] \
            ) \
         ).astype('float64')
    imdb_ratings_filtered.loc[:,'votecat'] \
        = imdb_ratings_filtered[['numVotes']].apply(lambda x: \
            sum([(x>rate_stats[1] - 2*rate_stats[3]) \
                ,(x>rate_stats[1] - rate_stats[3]) \
                ,(x>rate_stats[1]) \
                ,(x>rate_stats[1] + rate_stats[3]) \
                ,(x>rate_stats[1] + 2*rate_stats[3])] \
            ) \
         ).astype('float64')
    imdb_ratings_filtered.loc[:,'score'] \
        = imdb_ratings_filtered[['ratecat','votecat']].apply(lambda x: \
            sum(x[['ratecat','votecat']]) ,axis='columns' \
        )
    
    rate_dtype = {'numVotes':'int64','ratecat':'int64' ,'votecat':'int64' ,'score':'int64'}
    imdb_ratings_filtered_dtyped = imdb_ratings_filtered.astype(rate_dtype)
    
    return imdb_ratings_filtered_dtyped

In [53]:
Timenow()

Current Time = 01:06:49


In [54]:
#dtyped region dataframe
def Akas_dtype():
    imdb_akas=pd.read_csv(os.path.join(path,datapath,"imdb/title.akas.tsv/title.akas.tsv") , sep='\t', header=0 )
    #imdb_akas_copy = imdb_akas.copy()
    #imdb_akas=imdb_akas_copy.copy()
    #imdb_akas
    col_akas = imdb_akas.dtypes.to_dict()
    col_akas["titleId"]='object'
    col_akas['ordering']='int64'
    col_akas['title']='object'
    col_akas['region']='str'
    col_akas['language']='str'
    col_akas['types']='category'
    col_akas['attributes']='object'
    col_akas['isOriginalTitle']='bool'
    #col_akas
    imdb_akas_dtyped = imdb_akas.astype(col_akas)
    imdb_akas_cols = ['titleId','region','language']
    imdb_akas_dtyped = imdb_akas_dtyped[imdb_akas_cols].set_index(keys='titleId')
    #imdb_akas_dtyped
    
    return imdb_akas_dtyped

In [55]:
#feature expansion
def Region_count(dfX):
    #dfX = imdb_akas_dtype0_filtered
    df01 = dfX.groupby(dfX.index)
    df02 = pd.DataFrame(df01.agg(list))
    df02['regioncount'] = df02['region'].apply(len)
    
    return df02
    #245 values imdb_akas_movies.region.value_counts()
    #102 values imdb_akas_movies.language.value_counts() can ignore this
        #, since region has more values

In [22]:
Timenow()

Current Time = 20:21:19


/Volumes/lin/MachineLearning/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/Volumes/lin/MachineLearning/env/lib/python3.7/site-packages/ipykernel_launcher.py:16: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  app.launch_new_instance()


,averageRating,numVotes,ratecat,votecat,score
tconst,,,,,
tt0000417,8.2,40082,4,2,6
tt0000439,7.3,15661,3,2,5
tt0004972,6.4,20887,2,2,4
tt0006864,7.8,13116,3,2,5
tt0010323,8.1,51648,4,2,6
tt0012349,8.3,101632,4,3,7
tt0013442,7.9,82878,3,2,5
tt0014429,8.1,16799,4,2,6
tt0015324,8.2,35137,4,2,6


In [56]:
#feature expansion
def Regions_expand(dfX):
    '''akas_filtered = Ratings_netflix_filter(dfX ,0)
    cols_1hot = pd.DataFrame(np.ravel(akas_filtered.region.unique() ,1) ,columns=['tconst'])
    cols_1hot.set_index(keys=['tconst'] ,drop=True ,inplace=True)'''
    #cols_1hot

    akas_filtered = dfX.join(ratings.drop(columns=ratings.columns ,inplace=False) ,how='inner')
    region_counts = Region_count(akas_filtered)
    cols = []
    num = '{x:n}'
    for x in range(0 ,region_counts.region.str.len().max()):
        cols.append('region'+ num.format(x=x))

    df07 = pd.DataFrame(region_counts.region.tolist() ,columns=cols  ,index=[region_counts.index])
    #df07 = pd.DataFrame(cols_1hot.index.tolist() )
    df08 = df07.stack().reset_index()[0]
    #df09 = df08.loc[df08!='\\N']
    df09 = df08
    df0freq = df09.value_counts()
    #print(df0freq ,'\n' ,type(df0freq))
    cutoff = sum(df0freq)*0.01
    df0infreq = df0freq[df0freq<cutoff].index
    df010 = df09.replace(df0infreq ,'Other') 
    df011 = list(df010.unique())
    df0col = pd.DataFrame(columns=df011).astype('bool')
    
    df020 = region_counts['region'].replace(df0infreq ,'Other')
    df01concat = [df020 ,df0col]

    df01uniq = pd.concat(df01concat ,axis=1)
    
    return df01uniq ,df011

In [57]:
#regions_expanded ,df011 = Expanded_regions(Akas_dtyped())
#regions_expanded
#regions_expanded.dtypes

In [24]:
Timenow()

Current Time = 20:22:57


In [58]:
#exclusivley called by Region_1hot
def Match_region(dfX ,x ,Xcol ,col): #axis=index
    colname = x.name
    i = list(dfX.index)
    colval = dfX[col].loc[i]
    #colval = df.region.loc[i]
    return colval.apply(lambda f: colname in f)

In [ ]:
del ratings

In [44]:
del basicsC
del genresC
del ratingsC

In [39]:
#reduce memory
basics = basics.join(ratings.drop(columns=ratings.columns ,inplace=False) ,how='inner')
basics.dropna(axis='index' ,how='any' ,inplace=True)
basics

,titleType,primaryTitle,isAdult,startYear,endYear,runtimeMinutes,genres,runtime_cat
tconst,,,,,,,,
tt0000417,short,A Trip to the Moon,False,1902,0,13,"Action,Adventure,Comedy",0-30
tt0000439,short,The Great Train Robbery,False,1903,0,11,"Action,Crime,Short",0-30
tt0004972,movie,The Birth of a Nation,False,1915,0,195,"Drama,History,War",Long
tt0006864,movie,Intolerance: Love's Struggle Throughout the Ages,False,1916,0,163,"Drama,History",120-180
tt0010323,movie,The Cabinet of Dr. Caligari,False,1920,0,76,"Fantasy,Horror,Mystery",60-90
tt0012349,movie,The Kid,False,1921,0,68,"Comedy,Drama,Family",60-90
tt0013442,movie,Nosferatu,False,1922,0,94,"Fantasy,Horror",90-120
tt0014429,movie,Safety Last!,False,1923,0,74,"Action,Comedy,Thriller",60-90
tt0015324,movie,Sherlock Jr.,False,1924,0,45,"Action,Comedy,Romance",30-60


In [59]:
#custom 1hot encoder
def Region_1hot(dfX ,dfXcol ,col):
    df01hot = dfX.apply(lambda x: Match_region(dfX ,x ,dfXcol ,col)
                       if x.name in dfXcol else x
                       ,axis='index'
    #                   ,axis='columns'
    #                  ,result_type='reduce')
                             )
    #    col_dtypes = {'region':'list','language':'list'}
    #    df01hot = df01hot.astype(col_dtypes)
    return df01hot

In [60]:
#final pipeline
def Region(dfX):
    regions ,regionsscols = Regions_expand(dfX)
    dfY = Region_1hot(regions ,regionsscols ,'region')
    
    return dfY

In [51]:
basics.dropna(axis='index' ,how='any').index

Index(['tt0000417', 'tt0000439', 'tt0004972', 'tt0006864', 'tt0010323',
       'tt0012349', 'tt0013442', 'tt0014429', 'tt0015324', 'tt0015648',
       ...
       'tt9243946', 'tt9253866', 'tt9398640', 'tt9412098', 'tt9419834',
       'tt9425132', 'tt9432978', 'tt9495224', 'tt9561862', 'tt9573980'],
      dtype='object', name='tconst', length=8221)

In [43]:
#reduce memory
ratings = ratings.join(basics.drop(columns=basics.columns ,inplace=False) ,how='inner')
ratings.dropna(axis='index' ,how='any' ,inplace=True)
ratings

,averageRating,numVotes,ratecat,votecat,score
tconst,,,,,
tt0000417,8.2,40082,4,2,6
tt0000439,7.3,15661,3,2,5
tt0004972,6.4,20887,2,2,4
tt0006864,7.8,13116,3,2,5
tt0010323,8.1,51648,4,2,6
tt0012349,8.3,101632,4,3,7
tt0013442,7.9,82878,3,2,5
tt0014429,8.1,16799,4,2,6
tt0015324,8.2,35137,4,2,6


In [69]:
Timenow()

Current Time = 01:21:05


In [70]:
def CombineAll():
    basics = Basics_dtype()
    genres = Genre(Basics_dtype())
    regions = Region(Akas_dtype())
    
    #reduce memory
    basics = basics.join(ratings.drop(columns=ratings.columns ,inplace=False) ,how='inner')
    basics.dropna(axis='index' ,how='any' ,inplace=True)
    genres = genres.join(ratings.drop(columns=ratings.columns ,inplace=False) ,how='inner')
    genres.dropna(axis='index' ,how='any' ,inplace=True)
    
    genres['genrecounts'] = genres.genres.str.len()
    regions['regioncounts'] = regions.region.str.len()

    bcols = ['titleType','startYear','endYear','runtimeMinutes','runtime_cat']
    df0 = basics[bcols].join(genres.iloc[: ,1:] ,how='inner' ,rsuffix='_genre')
    df1 = df0.join(ratings ,how='inner')
    df2 = df1.join(regions.iloc[: ,1:] ,how='inner' ,rsuffix='_region')
    
    return df2

In [71]:
data = CombineAll()
del ratings
data

WENT TO EXCEPT 1


/Volumes/lin/MachineLearning/env/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()
/Volumes/lin/MachineLearning/env/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,titleType,startYear,endYear,runtimeMinutes,runtime_cat,Documentary,Short,Animation,Comedy,Romance,...,NO,GB,DK,\N_region,AU,RU,DE,IN,SK,regioncounts
tt0032138,movie,1939,0,102,90-120,False,False,False,False,False,...,True,True,True,True,True,True,True,False,False,54
tt0033467,movie,1941,0,119,90-120,False,False,False,False,False,...,False,True,True,True,True,True,True,True,True,61
tt0034583,movie,1942,0,102,90-120,False,False,False,False,True,...,False,True,True,True,False,False,False,False,False,41
tt0038650,movie,1946,0,130,120-180,False,False,False,False,False,...,True,True,True,True,False,True,True,False,True,55
tt0047396,movie,1954,0,112,90-120,False,False,False,False,False,...,True,True,True,True,False,True,True,False,True,58
tt0050083,movie,1957,0,96,90-120,False,False,False,False,False,...,True,True,True,True,False,True,True,True,True,56
tt0052357,movie,1958,0,128,120-180,False,False,False,False,True,...,False,True,True,True,False,True,True,False,False,58
tt0054215,movie,1960,0,109,90-120,False,False,False,False,False,...,False,True,True,True,False,True,False,True,False,46
tt0057012,movie,1964,0,95,90-120,False,False,False,True,False,...,True,True,True,True,True,True,True,False,True,68
tt0060196,movie,1966,0,161,120-180,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,76


In [72]:
Timenow()

Current Time = 01:24:43


In [34]:
#to return error #to return error #to return error #to return error #to return error #to return error 
#forceError['doesnt_exist']

In [73]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print (data.dtypes)


titleType         category
startYear            int64
endYear              int64
runtimeMinutes       int64
runtime_cat       category
Documentary           bool
Short                 bool
Animation             bool
Comedy                bool
Romance               bool
Other                 bool
Action                bool
Drama                 bool
Fantasy               bool
Horror                bool
Biography             bool
Music                 bool
Crime                 bool
Family                bool
Adventure             bool
Mystery               bool
\N                    bool
Sci-Fi                bool
Thriller              bool
Adult                 bool
genrecounts          int64
averageRating      float64
numVotes             int64
ratecat              int64
votecat              int64
score                int64
HU                    bool
UA                    bool
SE                    bool
AR                    bool
PL                    bool
US                    bool
G

In [ ]:

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [50]:
dftemp = Basics_genre_expand(Basics_dtype())
dftemp1 = pd.DataFrame(dftemp[0])
dftemp1

KeyboardInterrupt: 

In [53]:
dftemp1 = pd.DataFrame(dftemp[0])
dftemp1

,genres,Documentary,Short,Animation,Comedy,Romance,Other,Action,Drama,Fantasy,...,Biography,Music,Crime,Family,Adventure,Mystery,\N,Sci-Fi,Thriller,Adult
tt0000001,"Documentary,Short",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0000002,"Animation,Short",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0000003,"Animation,Comedy,Romance",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0000004,"Animation,Short",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0000005,"Comedy,Short",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0000006,Short,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0000007,"Short,Sport",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0000008,"Documentary,Short",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0000009,Romance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0000010,"Documentary,Short",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#1hot the categories dtypes

In [ ]:
#use PCA to remove unimportant data, maybe the one that clusters data

In [ ]:
#to return error #to return error #to return error #to return error #to return error #to return error 
forceError['doesnt_exist']

In [ ]:
with pd.option_context('display.max_rows', 10, 'display.max_columns', None):  # more options can be specified also
    print (df01hot)
#maybe also get the region counts released per movie

In [60]:
regions = Region(Akas_dtype())
regions

/Volumes/lin/MachineLearning/env/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,region,Other,GB,MX,UY,GR,ES,FI,FR,SE,...,RO,TR,AR,BG,HR,LT,EE,AU,PE,IL
tt0000417,"[CSHH, GB, MX, UY, GR, ES, FI, FR, SE, US, \N,...",False,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
tt0000439,"[IT, \N, JP, RU, FI, FR, CZ, IR, UA, CA, HU, N...",False,False,False,False,False,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
tt0004972,"[JP, FR, BR, \N, FI, DE, RO, TR, ES, AR, DK, X...",False,False,True,True,True,True,True,True,True,...,True,True,True,True,False,False,False,False,False,False
tt0006864,"[BR, US, \N, PL, ES, DE, PT, AR, US, HU, RS, D...",False,False,True,True,False,True,True,True,True,...,True,False,True,True,False,False,False,False,False,False
tt0010323,"[PT, BG, SE, CA, UA, DE, TR, PL, HU, RU, RS, F...",False,True,True,False,True,True,True,True,True,...,True,True,True,True,True,False,False,False,False,False
tt0012349,"[FR, IR, AR, LT, US, HR, \N, GR, IR, CZ, HU, D...",False,False,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,False,False
tt0013442,"[GB, PA, FR, HU, PL, \N, GB, XWW, XWW, \N, ES,...",False,True,True,False,True,True,True,True,True,...,True,False,True,True,False,True,True,True,False,False
tt0014429,"[SE, DK, UY, RU, DE, DE, GR, HU, HU, SI, FI, B...",False,False,False,True,True,True,True,True,True,...,False,False,False,False,False,False,True,False,False,False
tt0015324,"[RO, DE, UA, RU, FR, PL, IT, PT, CZ, FI, RS, S...",False,False,False,False,False,True,True,True,True,...,True,False,True,False,False,False,False,False,False,False
tt0015648,"[MX, GR, HU, SUHH, IN, VE, RS, CA, FI, XWW, US...",False,True,True,False,True,True,True,True,True,...,True,True,True,True,True,True,False,True,True,False


In [61]:
Timenow()

Current Time = 21:33:42


In [19]:
Timenow()

Current Time = 13:34:50


In [20]:
ratings = Ratings_expand(Ratings_netflix_filter(Ratings_dtype() ,1))
ratings

/Volumes/lin/MachineLearning/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/Volumes/lin/MachineLearning/env/lib/python3.7/site-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.


,averageRating,numVotes,ratecat,votecat,score
tconst,,,,,
tt0000417,8.2,40082.0,4.0,2.0,6.0
tt0000439,7.3,15661.0,3.0,2.0,5.0
tt0004972,6.4,20887.0,2.0,2.0,4.0
tt0006864,7.8,13116.0,3.0,2.0,5.0
tt0010323,8.1,51648.0,4.0,2.0,6.0
tt0012349,8.3,101632.0,4.0,3.0,7.0
tt0013442,7.9,82878.0,3.0,2.0,5.0
tt0014429,8.1,16799.0,4.0,2.0,6.0
tt0015324,8.2,35137.0,4.0,2.0,6.0


In [21]:
Timenow()

Current Time = 13:36:29


In [ ]:
ratings.dtypes

In [ ]:
def Principals_dtype():
    imdb_principals = pd.read_csv(os.path.join(path,datapath,"imdb/title.principals.tsv/title.principals.tsv"), sep='\t', header=0)                                   
    
    col_principals = imdb_principals.dtypes.to_dict()
    col_principals['tconst']='object'
    col_principals['ordering']='int64'
    col_principals['nconst']='str'
    col_principals['category']='category'
    col_principals['job']='category'
    col_principals['characters']='str'
    col_principals
    
    imdb_principals_dtyped = imdb_principals.astype(col_principals)
    imdb_principals_dtyped.set_index(keys='tconst' ,drop=True ,inplace=True)
    
    return imdb_principals_dtyped

In [ ]:
#to return error #to return error #to return error #to return error #to return error #to return error 
forceError['doesnt_exist']

In [ ]:
def Principals_dropna(dfX): #exclusively calls Principals_dtype()
    Principals_dtyped = dfX   
    df = Principals_dtyped[['nconst','category']]
    
    director = df[['nconst']].loc[df.category=='director']
    writer = df[['nconst']].loc[df.category=='writer']
    actor = df[['nconst']].loc[df.category=='actor']
    
    principals0 = df.join(director ,rsuffix='_director' ,how='left')
    principals1 = principals0.join(writer ,rsuffix='_writer' ,how='left')
    principals2 = principals1.join(actor ,rsuffix='_actor' ,how='left')
    principals3 = principals2[['nconst_director','nconst_writer' ,'nconst_actor']].drop_duplicates(inplace=False)
    principals3.dropna(inplace=True)
    principals3.reset_index(inplace=True)
    principals4 = principals3[['tconst']].drop_duplicates()
    principals5 = principals4[['tconst']].set_index(keys='tconst' ,inplace=False)
    
    filtered = dfX.join(principals5 ,how='inner')  
    return filtered

In [ ]:
#has duplicates imdb_akas.groupby(['titleId']).titleId.value_counts()
#no duplicates imdb_basics.groupby(['tconst']).tconst.value_counts()
#has duplicates imdb_principals.groupby(['tconst']).tconst.value_counts()

In [63]:
regions.GB.value_counts()

False    4042
True     3506
Name: GB, dtype: int64

In [ ]:
principals_filtered = Principals_dropna(Ratings_netflix_filter(Principals_dtype()))
principals_filtered

In [ ]:
#df = principals_filtered.loc[principals_filtered.index=='tt0054130']
#df.groupby([df.index,'nconst'])['nconst'].apply(lambda x: x.str.len()+1)

,titleType,startYear,endYear,runtimeMinutes,runtime_cat,genres,Documentary,Short,Animation,Comedy,...,Biography_genre,Music_genre,Crime_genre,Family_genre,Adventure_genre,Mystery_genre,\N_genre,Sci-Fi_genre,Thriller_genre,Adult_genre
tconst,,,,,,,,,,,,,,,,,,,,,
tt0000417,short,1902,0,13,0-30,"Action,Adventure,Comedy",False,False,False,True,...,False,False,False,False,True,False,False,False,False,False
tt0000439,short,1903,0,11,0-30,"Action,Crime,Short",False,True,False,False,...,False,False,True,False,False,False,False,False,False,False
tt0004972,movie,1915,0,195,Long,"Drama,History,War",False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
tt0006864,movie,1916,0,163,120-180,"Drama,History",False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
tt0010323,movie,1920,0,76,60-90,"Fantasy,Horror,Mystery",False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
tt0012349,movie,1921,0,68,60-90,"Comedy,Drama,Family",False,False,False,True,...,False,False,False,True,False,False,False,False,False,False
tt0013442,movie,1922,0,94,90-120,"Fantasy,Horror",False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
tt0014429,movie,1923,0,74,60-90,"Action,Comedy,Thriller",False,False,False,True,...,False,False,False,False,False,False,False,False,True,False
tt0015324,movie,1924,0,45,30-60,"Action,Comedy,Romance",False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [79]:
df2.dtypes

titleType            category
startYear               int64
endYear                 int64
runtimeMinutes          int64
runtime_cat          category
genres                 object
Documentary              bool
Short                    bool
Animation                bool
Comedy                   bool
Romance                  bool
Other                    bool
Action                   bool
Drama                    bool
Fantasy                  bool
Horror                   bool
Biography                bool
Music                    bool
Crime                    bool
Family                   bool
Adventure                bool
Mystery                  bool
\N                       bool
Sci-Fi                   bool
Thriller                 bool
Adult                    bool
averageRating         float64
numVotes                int64
ratecat                 int64
votecat                 int64
score                   int64
Documentary_genre        bool
Short_genre              bool
Animation_

In [ ]:
principals_filtered

In [75]:
genres.iloc[: ,1:]

,Documentary,Short,Animation,Comedy,Romance,Other,Action,Drama,Fantasy,Horror,Biography,Music,Crime,Family,Adventure,Mystery,\N,Sci-Fi,Thriller,Adult
tt0000417,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False
tt0000439,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False
tt0004972,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
tt0006864,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
tt0010323,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,True,False,False,False,False
tt0012349,False,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False
tt0013442,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False
tt0014429,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False
tt0015324,False,False,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
tt0015648,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False


In [ ]:
principals_filtered.loc[principals_filtered.index=='tt0054130']

In [ ]:
principals_filtered.loc[principals_filtered.category=='director']

In [ ]:
principals_filtered[['nconst','category']]

In [ ]:
def join_df(leftdf ,leftindex ,rightdf ,rightindex):
    left = leftdf.set_index([leftindex],)
    right = rightdf.set_index([rightindex])
    return left.join(right ,how='inner')

In [ ]:
import random as rd

In [ ]:
def Director(dfX):
    principals_directors = dfX.loc[dfX.category=='director']
    principals_directors.head()
    #unique nconst to tconst combination
    principals_directors.groupby([principals_directors.index,'nconst']).nconst.value_counts()
    #unique nconst to tconst combination

    df1 = pd.DataFrame(principals_directors[['nconst','category']]).astype(str)
    #.sample(n=1000))#.astype('str')#.reset_index()
    #imdb_principals_groupedDirectors = df1.groupby(df1.index).agg(lambda x: [','.join(set(x))])
    #imdb_principals_groupedDirectors
    df10 = df1.reset_index()
    df11 = df10.groupby(df10['nconst'])
    df12directors = pd.DataFrame(df11[['nconst','tconst']].agg(list)).reset_index(drop=True)
    df12directors['directorcount'] = df12directors.nconst.apply(len)
    df12directors['director'] = df12directors['nconst'].str[0]
    df12directors.drop(columns=['nconst'] ,inplace=True)
    df12directors.set_index(keys='tconst' ,inplace=True ,drop=True)
    
    return df12directors

In [ ]:
principals_directors = Director(principals_filtered)
principals_directors

In [ ]:
def Writer(dfX):
    principals_writers = dfX.loc[dfX.category=='writer']
    principals_writers.head()
    #unique nconst to tconst combination
    principals_writers.groupby([principals_writers.index,'nconst'])['nconst'].value_counts()
    #unique nconst to tconst combination

    df2 = pd.DataFrame(principals_writers[['nconst','category']]).astype(str)
    #.sample(n=1000))#.astype('str')#.reset_index()
    #imdb_principals_groupedDirectors = df1.groupby(df1.index).agg(lambda x: [','.join(set(x))])
    #imdb_principals_groupedDirectors
    df20 = df2.reset_index()
    df21 = df20.groupby(df20['nconst'])
    df22writers = pd.DataFrame(df21[['nconst','tconst']].agg(list)).reset_index(drop=True)
    df22writers['writercount'] = df22writers.nconst.apply(len)
    df22writers['writer'] = df22writers['nconst'].str[0]
    df22writers.drop(columns=['nconst'] ,inplace=True)
    df22writers.set_index(keys='tconst' ,inplace=True ,drop=True)
    
    return df22writers

In [ ]:
principals_writers = Writer(principals_filtered)
principals_writers

In [ ]:
def Actor(dfX):
    principals_actors = principals_filtered.loc[principals_filtered.category=='actor']
    principals_actors.head()
    #unique nconst to tconst combination
    principals_actors.groupby([principals_actors.index,'nconst'])['nconst'].value_counts()
    #unique nconst to tconst combination

    df3 = pd.DataFrame(principals_actors[['nconst','category']]).astype(str)
    #.sample(n=1000))#.astype('str')#.reset_index()
    #imdb_principals_groupedDirectors = df1.groupby(df1.index).agg(lambda x: [','.join(set(x))])
    #imdb_principals_groupedDirectors
    df30 = df3.reset_index()
    df31 = df30.groupby(df30['nconst'])
    df32actors = pd.DataFrame(df31[['nconst','tconst']].agg(list)).reset_index(drop=True)
    df32actors['actorcount'] = df32actors.nconst.apply(len)
    df32actors['actor'] = df32actors['nconst'].str[0]
    df32actors.drop(columns=['nconst'] ,inplace=True)
    df32actors.set_index(keys='tconst' ,inplace=True ,drop=True)
    
    return df32actors

In [ ]:
principals_actors = Actor(principals_filtered)
principals_actors

In [ ]:
#list of generated dataframes:
#    principals_actors ,principals_writers ,principals_directors ,principals_filtered ,regions_1hot
#    ,ratings_expanded ,basics_netflix_uniq ,imdb_basics_expanded

In [ ]:
# now fill in blank actorcount, moviecount, writercount, and region with "Other"
# maybe df32actors is not important? hard to correlate actor popularity within dataset

In [ ]:
imdb_basics_expanded.join(principals_actors ,how='inner')

In [ ]:
#to return error
forceError['doesnt_exist']

In [90]:
basics

""
tconst
tt0000001
tt0000002
tt0000003
tt0000004
tt0000005
tt0000006
tt0000007
tt0000008
tt0000009


In [ ]:
#netflix stuff

In [60]:
dfY = ratings.copy()
dfY.drop(dfY.columns ,axis='columns' ,inplace=True)
dfY

""
tconst
tt0000417
tt0000439
tt0004972
tt0006864
tt0010323
tt0012349
tt0013442
tt0014429
tt0015324


In [61]:
del dfY

In [ ]:
def join_df(leftdf ,leftindex ,rightdf ,rightindex):
    left = leftdf.set_index([leftindex],)
    right = rightdf.set_index([rightindex])
    return left.join(right ,how='inner')

In [ ]:
#imdb1 = join_df(imdb_akas ,'titleId' ,imdb_basics ,'tconst')

In [ ]:
#imdb = join_df(imdb1 ,'titleId' ,imdb_basics ,'tconst')

In [ ]:
netflix_raw=pd.read_csv(os.path.join(path,datapath,"netflix_titles.csv"))

In [84]:
def reduce_memory(dfX ,dfY):
    dfReduce = dfX.join(dfY)
    
    return dfReduce.index

In [85]:
dfY = ratings.drop(columns=ratings.columns ,axis='columns' ,inplace=False)
dfY

""
tconst
tt0000417
tt0000439
tt0004972
tt0006864
tt0010323
tt0012349
tt0013442
tt0014429
tt0015324


In [87]:
ratings.index

Index(['tt0000417', 'tt0000439', 'tt0004972', 'tt0006864', 'tt0010323',
       'tt0012349', 'tt0013442', 'tt0014429', 'tt0015324', 'tt0015648',
       ...
       'tt9253866', 'tt9398640', 'tt9412098', 'tt9419834', 'tt9425132',
       'tt9432978', 'tt9495224', 'tt9561862', 'tt9573980', 'tt9906260'],
      dtype='object', name='tconst', length=8419)

In [75]:
dfReduce = basics.join(dfY ,how='inner')
dfReduce

TypeError: 'NoneType' object is not iterable

In [ ]:
netflix_raw.head()

In [ ]:
netflix_raw.loc[netflix_raw.release_year>2015].release_year.value_counts()

In [ ]:
#scrap work

In [ ]:
#manually filling NaN exercise
'''avgRateFill = ratings['averageRating'].loc[ratings.averageRating.notnull()].agg('average')
avgVoteFill = ratings['numVotes'].loc[ratings.numVotes.notnull()].agg('average')
print( avgRateFill ,' : ' ,avgVoteFill)

fillvals = {'averageRating':avgRateFill ,'numVotes':avgVoteFill}
ratings.fillna(value=fillvals ,inplace=True)'''

In [ ]:
#manually filling NaN exercise
'''ratings_std = ratings['averageRating'].agg('std')
votes_std = ratings['numVotes'].agg('std')
print('stds: ' ,ratings_std ,' : ' ,votes_std)'''

In [ ]:
string = ''
print(string)
string = 'asda'
print(string)

In [73]:
ratings

""
tconst
tt0000417
tt0000439
tt0004972
tt0006864
tt0010323
tt0012349
tt0013442
tt0014429
tt0015324


In [ ]:
string = 'abc'
print(type(string))
str_ser = pd.Series(string)
print(type(str_ser))
if str_ser.isin(['abc','cbc']).bool():
    print('True')
else:
    print('whack')

In [ ]:
tag = soup.find(class_="credit_summary_item")
for child in tag.stripped_strings:
    print(child ,type(child) )#,repr(child)[:5] ,type(repr(child)))
    chil = pd.Series([child])
    print(chil ,type(chil))

In [ ]:
title_wrapper = soup.find('div',class_="title_wrapper")
title_wrapper_stripped = title_wrapper.stripped_strings
title = list(title_wrapper_stripped)[0]
title

In [ ]:
regions = regions_1hot.copy()

In [151]:
regions.dtypes

region    object
Other       bool
GB          bool
MX          bool
UY          bool
GR          bool
ES          bool
FI          bool
FR          bool
SE          bool
US          bool
\N          bool
BR          bool
PT          bool
HU          bool
RS          bool
RU          bool
SI          bool
IT          bool
NO          bool
DE          bool
JP          bool
CZ          bool
PL          bool
DK          bool
UA          bool
CA          bool
RO          bool
TR          bool
AR          bool
BG          bool
HR          bool
LT          bool
EE          bool
AU          bool
PE          bool
IL          bool
dtype: object

In [ ]:
del regions_1hot

In [ ]:
title_wrapper_stripped = title_wrapper.stripped_strings
rated = list(title_wrapper_stripped)[4]
rated

In [ ]:
imdbRating = soup.find(class_="imdbRating")
imdbRating_stripped = imdbRating.stripped_strings
rating = list(imdbRating_stripped)[0]
rating

In [ ]:
imdbRating_stripped = imdbRating.stripped_strings
review_count = list(imdbRating_stripped)[3]
review_count

In [ ]:
imdbRating_stripped = imdbRating.stripped_strings
user_review_count = list(imdbRating_stripped)[4]
user_review_count

In [ ]:
imdbRating_stripped = imdbRating.stripped_strings
critic_review_count = list(imdbRating_stripped)[5]
critic_review_count

In [ ]:
basics ,basicscols ,'genres'

In [ ]:
def Genre_1hot(dfX ,dfXcol ,col):
    df01hot = dfX.apply(lambda x: Match_genre(dfX ,x ,dfXcol ,col)
                       if x.name in dfXcol else x
    #                   ,axis='index'
    #                   ,axis='columns'
    #                  ,result_type='reduce')
                             )
    #    col_dtypes = {'region':'list','language':'list'}
    #    df01hot = df01hot.astype(col_dtypes)
    return df01hot

In [ ]:
def Match_genre(dfX ,x ,Xcol ,col): #axis=index
    colname = x.name
    i = list(dfX.index)
    colval = dfX[col].loc[i]
    #colval = df.region.loc[i]
    return colval.apply(lambda f: colname in f)

In [ ]:
regions_expanded